In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt

# import seaborn
import seaborn as sns
# settings for seaborn plotting style
sns.set(color_codes=True)
# settings for seaborn plot sizes
sns.set(rc={'figure.figsize':(12,6)})

from nfl_data_functions import normalize_data
from nfl_data_functions import split_data
from nfl_data_functions import tune_rf_grid
from nfl_data_functions import tune_gb_grid
from nfl_data_functions import tune_lr_grid
from nfl_data_functions import tune_svm_grid
from nfl_data_functions import tune_knn_grid
from nfl_data_functions import tune_dt_grid
from nfl_data_functions import tune_nn_grid
from nfl_data_functions import tune_models
from nfl_data_functions import get_seasons
from nfl_data_functions import get_team_results
from nfl_data_functions import get_team_results_rolling_avg
from nfl_data_functions_evan import get_team_results_weighted
from nfl_data_functions_evan import get_data_home_away_avg_joined
from nfl_data_functions_evan import get_team_results_expon_rolling_avg
from nfl_data_functions_evan import get_next_week_x_train
data = pd.read_csv('data.csv')
df = pd.read_csv('df.csv')

In [44]:
# All you have to change is the seasons for which seasons you want and the weighting parameters
# or rolling average call
seasons = range(2021,2022)   # Specifiy what seasons you want. This gets 2015-2021

seasons_df = get_seasons(seasons, df)   # Dataframe with seasons 2015-2021
team_names, team_results = get_team_results(seasons_df)
team_results_weighted = get_team_results_expon_rolling_avg(team_names, team_results,10)

# uncomment the line below to use the rolling average (see function for details in nfl_data_functions.py)
# team_results_weighted = get_team_results_rolling_avg(team_names, team_results)

df_new = get_data_home_away_avg_joined(data, team_results_weighted, seasons=seasons)

/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:364: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['date', 'home', 'opponent', 'team'], dtype='object')
  era_col = team_results[name].ewm(span=span_avg, adjust=False).mean()
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:364: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['date', 'home', 'opponent', 'team'], dtype='object')
  era_col = team_results[name].ewm(span=span_avg, adjust=False).mean()
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:364: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future vers

In [45]:
team_results_weighted["Eagles"]

,date,opponent,team,home,first_downs_opp,first_downs,third_downs_opp,third_downs,fourth_downs_opp,fourth_downs,passing_yards_opp,passing_yards,rushing_yards_opp,rushing_yards,total_yards_opp,total_yards,sacks_opp,sacks,rushing_attempts_opp,rushing_attempts,fumbles_opp,fumbles,int_opp,int,turnovers_opp,turnovers,penalties_opp,penalties,drives_opp,drives,def_st_td_opp,def_st_td,possession_opp,possession,score_opp,score,score_diff,winner,third_downs_rate_opp,third_downs_attempts_opp,third_downs_rate,third_downs_attempts,fourth_downs_rate_opp,fourth_downs_attempts_opp,fourth_downs_rate,fourth_downs_attempts,complete_passes_opp,pass_completion_rate_opp,pass_attempt_opp,complete_passes,pass_completion_rate,pass_attempt,pressures_opp,pressures,penalties_yards_opp,penalties_yards,redzone_sucess_opp,redzone_rate_opp,redzone_attempts_opp,redzone_sucess,redzone_rate,redzone_attempts,season,day_number
10162,2021-09-12,Falcons,Eagles,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0,6947.000000
10185,2021-09-19,49ers,Eagles,Y,19.000000,24.000000,3.000000,6.000000,3.000000,0.000000,136.000000,261.000000,124.000000,173.000000,260.000000,434.000000,3.000000,1.000000,26.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,14.000000,12.000000,11.000000,0.000000,0.000000,1738.000000,1862.000000,6.000000,32.000000,26.000000,1.000000,0.214286,14.000000,0.461538,13.000000,0.214286,14.000000,0.000000,2.000000,21.000000,0.600000,35.000000,27.000000,0.771429,35.000000,28.000000,3.000000,99.000000,89.000000,0.000000,0.000000,2.000000,3.000000,1.000000,3.000000,2021.0,6948.272727
10238,2021-09-27,Cowboys,Eagles,N,19.727273,22.909091,3.545455,5.818182,2.636364,0.909091,145.636364,245.727273,122.727273,169.000000,268.363636,414.727273,2.454545,1.181818,28.181818,30.636364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.727273,12.909091,11.454545,10.636364,0.000000,0.000000,1802.727273,1797.272727,8.000000,28.181818,20.181818,0.818182,0.253247,14.000000,0.453380,12.818182,0.357143,11.636364,0.075758,3.818182,21.181818,0.624242,34.090909,24.272727,0.722078,33.000000,22.909091,4.818182,91.909091,83.181818,0.363636,0.181818,2.000000,2.636364,0.909091,2.818182,2021.0,6950.768595
10252,2021-10-03,Chiefs,Eagles,Y,21.049587,20.925620,3.991736,5.487603,3.247934,0.743802,159.157025,256.140496,129.504132,149.909091,288.661157,406.049587,2.735537,1.330579,30.512397,27.247934,0.181818,0.000000,0.000000,0.363636,0.181818,0.363636,9.504132,12.925620,11.735537,10.884298,0.181818,0.181818,1856.413223,1743.586777,14.000000,26.876033,12.876033,0.669421,0.298111,13.636364,0.431553,12.669421,0.383117,11.702479,0.061983,3.487603,21.148760,0.657597,32.619835,24.404959,0.707341,34.090909,22.016529,8.123967,81.925620,83.694215,1.024793,0.294215,2.545455,2.520661,0.925620,2.669421,2021.0,6953.901578
10284,2021-10-10,Panthers,Eagles,N,22.858753,22.575507,4.902329,5.580766,2.657400,1.699474,179.492111,274.660406,142.321563,141.380165,321.813674,416.040571,2.419985,1.634110,30.782870,25.748310,0.148760,0.000000,0.181818,0.297521,0.330579,0.297521,9.048835,12.211871,11.056349,10.359880,0.148760,0.148760,1855.610819,1744.389181,19.090909,27.444027,8.353118,0.547708,0.407545,12.975207,0.443998,12.547708,0.313459,9.574756,0.141623,5.035312,21.667168,0.683488,32.143501,25.785875,0.699946,36.619835,19.286251,11.919609,75.393689,77.386176,1.747558,0.422539,2.991736,2.607814,0.848234,3.274981,2021.0,6957.737655
10305,2021-10-14,Buccaneers,Eagles,Y,21.793525,21.198142,4.920087,5.293354,3.083328,1.390479,175.584455,257.813059,136.263097,132.220135,311.847551,390.033194,2.525442,1.700635,30.640530,25.066799,0.121713,0.181818,0.694215,0.425244,0.815928,0.607062,9.039956,11.082440,11.591558,11.203538,0.121713,0.121713,1900.590670,1699.409330,18.892562,26.272386,7.379824,0.

In [53]:
# Drop all the columns that we don't need since thet are not numeric or predictive
df_new.dropna(inplace=True)
df_X = df_new.drop(['date','home','away','winner','score_home','score_away','score_diff','home_awayAvg','home_homeAvg','day_number_awayAvg','season_homeAvg','day_number_homeAvg','season_awayAvg'], axis=1)
Y = df_new['winner']
import random
random.seed(42)
# # Y = df_new['score_diff']   # for predicting score difference(spread)
# split data into train and test sets
# X_train, X_test, y_train, y_test = split_data(df_X, Y, test_size=0.2, random_state=42)
X_train_full, X_train, X_val, X_test, y_train_full, y_train, y_val, y_test = split_data(df_X, Y, 13)

In [54]:
results_dt = tune_dt_grid(X_train_full, y_train_full, X_test, y_test, criterion=['gini', 'entropy'], splitter=['best', 'random'], max_depth=[None, 1, 5, 10], min_samples_split=[2, 10], min_samples_leaf=[1, 2, 4])

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV]

In [52]:
previous_games = df_new.iloc[100:194]
testy_x = df_new.iloc[194:208]
# testy_x = testy_x.drop(['date','home','away','winner','score_home','score_away','score_diff','home_awayAvg','home_homeAvg','day_number_awayAvg','season_homeAvg','day_number_homeAvg','season_awayAvg'], axis=1)
# df_new.iloc[185:193]
next_homes = testy_x['home']
next_aways = testy_x['away']
len(testy_x.away)

14

In [83]:
df_new

,date,home,away,winner,score_home,score_away,score_diff,home_homeAvg,first_downs_opp_homeAvg,first_downs_homeAvg,third_downs_opp_homeAvg,third_downs_homeAvg,fourth_downs_opp_homeAvg,fourth_downs_homeAvg,passing_yards_opp_homeAvg,passing_yards_homeAvg,rushing_yards_opp_homeAvg,rushing_yards_homeAvg,total_yards_opp_homeAvg,total_yards_homeAvg,sacks_opp_homeAvg,sacks_homeAvg,rushing_attempts_opp_homeAvg,rushing_attempts_homeAvg,fumbles_opp_homeAvg,fumbles_homeAvg,int_opp_homeAvg,int_homeAvg,turnovers_opp_homeAvg,turnovers_homeAvg,penalties_opp_homeAvg,penalties_homeAvg,drives_opp_homeAvg,drives_homeAvg,def_st_td_opp_homeAvg,def_st_td_homeAvg,possession_opp_homeAvg,possession_homeAvg,score_opp_homeAvg,score_homeAvg,score_diff_homeAvg,winner_homeAvg,third_downs_rate_opp_homeAvg,third_downs_attempts_opp_homeAvg,third_downs_rate_homeAvg,third_downs_attempts_homeAvg,fourth_downs_rate_opp_homeAvg,fourth_downs_attempts_opp_homeAvg,fourth_downs_rate_homeAvg,fourth_downs_attempts_homeAvg,...,total_yards_awayAvg,sacks_opp_awayAvg,sacks_awayAvg,rushing_attempts_opp_awayAvg,rushing_attempts_awayAvg,fumbles_opp_awayAvg,fumbles_awayAvg,int_opp_awayAvg,int_awayAvg,turnovers_opp_awayAvg,turnovers_awayAvg,penalties_opp_awayAvg,penalties_awayAvg,drives_opp_awayAvg,drives_awayAvg,def_st_td_opp_awayAvg,def_st_td_awayAvg,possession_opp_awayAvg,possession_awayAvg,score_opp_awayAvg,score_awayAvg,score_diff_awayAvg,winner_awayAvg,third_downs_rate_opp_awayAvg,third_downs_attempts_opp_awayAvg,third_downs_rate_awayAvg,third_downs_attempts_awayAvg,fourth_downs_rate_opp_awayAvg,fourth_downs_attempts_opp_awayAvg,fourth_downs_rate_awayAvg,fourth_downs_attempts_awayAvg,complete_passes_opp_awayAvg,pass_completion_rate_opp_awayAvg,pass_attempt_opp_awayAvg,complete_passes_awayAvg,pass_completion_rate_awayAvg,pass_attempt_awayAvg,pressures_opp_awayAvg,pressures_awayAvg,penalties_yards_opp_awayAvg,penalties_yards_awayAvg,redzone_sucess_opp_awayAvg,redzone_rate_opp_awayAvg,redzone_attempts_opp_awayAvg,redzone_sucess_awayAvg,redzone_rate_awayAvg,redzone_attempts_awayAvg,season_awayAvg,day_number_awayAvg,week
0,2021-09-09,Buccaneers,Cowboys,1,31,29,2,Y,30.0,24.0,9.0,5.0,0.0,5.0,391.0,379.0,60.0,52.0,451.0,431.0,1.0,0.0,18.0,14.0,0.0,2.0,1.0,2.0,1.0,4.0,8.0,11.0,13.0,13.0,0.0,0.0,2067.0,1533.0,29.0,31.0,2.0,1.0,0.529412,17.0,0.454545,11.0,0.0,0.0,0.454545,11.0,...,451.0,0.0,1.0,14.0,18.0,2.0,0.0,2.0,1.0,4.0,1.0,11.0,8.0,13.0,13.0,0.0,0.0,1533.0,2067.0,31.0,29.0,-2.0,0.0,0.454545,11.0,0.529412,17.0,0.454545,11.0,0.0,0.0,32.0,0.64,50.0,42.0,0.724138,58.0,0.0,12.0,106.0,55.0,3.0,0.6,5.0,1.0,0.25,4.0,2021.0,6944.0,1
1,2021-09-12,Falcons,Eagles,0,6,32,-26,Y,24.0,19.0,6.0,3.0,0.0,3.0,261.0,136.0,173.0,124.0,434.0,260.0,1.0,3.0,31.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,12.0,11.0,12.0,0.0,0.0,1862.0,1738.0,32.0,6.0,-26.0,0.0,0.461538,13.0,0.214286,14.0,0.0,2.0,0.214286,14.0,...,434.0,3.0,1.0,26.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,14.0,12.0,11.0,0.0,0.0,1738.0,1862.0,6.0,32.0,26.0,1.0,0.214286,14.0,0.461538,13.0,0.214286,14.0,0.0,2.0,21.0,0.6,35.0,27.0,0.771429,35.0,28.0,3.0,99.0,89.0,0.0,0.0,2.0,3.0,1.0,3.0,2021.0,6947.0,1
2,2021-09-12,Bills,Steelers,0,16,23,-7,Y,16.0,22.0,4.0,8.0,0.0,8.0,177.0,254.0,75.0,117.0,252.0,371.0,2.0,3.0,21.0,25.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,8.0,11.0,11.0,1.0,0.0,1603.0,1997.0,23.0,16.0,-7.0,0.0,0.333333,12.0,0.444444,18.0,0.0,0.0,0.444444,18.0,...,252.0,3.0,2.0,25.0,21.0,1.0,0.0,0.0,0.0,1.0,0.0,8.0,5.0,11.0,11.0,0.0,1.0,1997.0,1603.0,16.0,23.0,7.0,1.0,0.444444,18.0,0.333333,12.0,0.444444,18.0,0.0,0.0,30.0,0.588235,51.0,18.0,0.5625,32.0,16.0,11.0,81.0,32.0,1.0,0.25,4.0,1.0,0.333333,3.0,2021.0,6947.0,1
3,2021-09-12,Bengals,Vikings,1,27,24,3,Y,24.0,20.0,6.0,3.0,2.0,3.0,336.0,217.0,67.0,149.0,403.0,366.0,3.0,5.0,22.0,36.0,1.0,0.0,0.0,0.0,1.0,0.0,12.0,3.0,13.0,13.0,0.0,0.0,2109.0,2091.0,24.0,27.0,3.0,1.0,0.375,16.0,0.214286,14.0,1.0,2.0,0.214286,14.0,...,403.0,5.0,3.0,36.0,22.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,12.0,13.0,13.0,0.0,0.0,2091.0,2109.0,27.0,24.0,-3.0,0

In [84]:
from sklearn.tree import DecisionTreeClassifier
def get_training_data_for_next_week(df_new, week_testing):  # must choose week 4 or laters!
    df_new["week"] = df_new["date"].apply(week_num)
    previous_games = df_new[df_new["week"] < week_testing].drop(["week"],axis=1)
    testing_week = df_new.query("week <= @week_testing and week > @week_testing - 1")
    next_homes = testing_week.home
    next_aways = testing_week.away
    true_wins = testing_week.winner
    x_train = get_next_week_x_train(previous_games, next_homes, next_aways)
    return x_train, true_wins

results_dict = {}
for i in range(4,18):
    x_train, true_wins = get_training_data_for_next_week(df_new, i)
    df_X = x_train.drop(['day_number_awayAvg','season_homeAvg','day_number_homeAvg','season_awayAvg'], axis=1)
    results_dt = tune_dt_grid(X_train_full, y_train_full, X_test, y_test, criterion=['gini', 'entropy'], splitter=['best', 'random'], max_depth=[None, 1, 5, 10], min_samples_split=[2, 10], min_samples_leaf=[1, 2, 4])
    params = results_dt["best_dt_params"]
    clf = DecisionTreeClassifier(criterion=params['criterion'], splitter=params['splitter'], max_depth=params['max_depth'], min_samples_split=params['min_samples_split'], min_samples_leaf=params['min_samples_leaf'])
    clf.fit(X_train_full, y_train_full)
    error = 1-np.mean(abs(true_wins - clf.predict(df_X)))
    results_dict[i] = error

/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV]

/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Pro

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV]

/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Pro

...

/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Pro

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV]

/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:443: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Pro

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV]

/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [86]:
# mean of all vlaues in dictionary
sum(results_dict.values())/len(results_dict)

0.5914584423512996

In [19]:
import datetime
# show just date and day_number_awayAvg
# make date col into datetime
df_new['date'] = pd.to_datetime(df_new['date'])
end_of_week_1 = datetime.datetime(2021, 9, 13)

In [39]:
df_new.iloc[0:20]
def week_num(date):
    # turn sting date into datetime
    end_of_week_1 = datetime.datetime(2021, 9, 13)
    if date <= end_of_week_1:
        return 1
    elif date <= end_of_week_1 + datetime.timedelta(days=7):
        return 2
    elif date <= end_of_week_1 + datetime.timedelta(days=14):
        return 3
    elif date <= end_of_week_1 + datetime.timedelta(days=21):
        return 4
    elif date <= end_of_week_1 + datetime.timedelta(days=28):
        return 5
    elif date <= end_of_week_1 + datetime.timedelta(days=35):
        return 6
    elif date <= end_of_week_1 + datetime.timedelta(days=42):
        return 7
    elif date <= end_of_week_1 + datetime.timedelta(days=49):
        return 8
    elif date <= end_of_week_1 + datetime.timedelta(days=56):
        return 9
    elif date <= end_of_week_1 + datetime.timedelta(days=63):
        return 10
    elif date <= end_of_week_1 + datetime.timedelta(days=70):
        return 11
    elif date <= end_of_week_1 + datetime.timedelta(days=77):
        return 12
    elif date <= end_of_week_1 + datetime.timedelta(days=84):
        return 13
    elif date <= end_of_week_1 + datetime.timedelta(days=91):
        return 14
    elif date <= end_of_week_1 + datetime.timedelta(days=98):
        return 15
    elif date <= end_of_week_1 + datetime.timedelta(days=105):
        return 16
    elif date <= end_of_week_1 + datetime.timedelta(days=112):
        return 17
    elif date <= end_of_week_1 + datetime.timedelta(days=119):
        return 18
    else:
        return 19
    # df_new["week"] = 	day_number_awayAvg

In [40]:
# df_new["date"].apply(week_num)
df_new["date"].apply(week_num)

0       1
1       1
2       1
3       1
4       1
       ..
280    19
281    19
282    19
283    19
284    19
Name: date, Length: 285, dtype: int64

In [63]:
previous_games = df_new.iloc[100:194]
testy_x = df_new.iloc[194:208]
# testy_x = testy_x.drop(['date','home','away','winner','score_home','score_away','score_diff','home_awayAvg','home_homeAvg','day_number_awayAvg','season_homeAvg','day_number_homeAvg','season_awayAvg'], axis=1)
# df_new.iloc[185:193]
next_homes = testy_x['home']
next_aways = testy_x['away']
len(testy_x.away)

14

In [64]:
next_homes

194       Vikings
195        Browns
196        Titans
197        Chiefs
198          Jets
199    Washington
200      Panthers
201        Texans
202       Broncos
203      Chargers
204       Bengals
205    Buccaneers
206       Packers
207     Cardinals
Name: home, dtype: object

In [5]:
testy_x

,date,home,away,winner,score_home,score_away,score_diff,home_homeAvg,day_number_homeAvg,first_downs_opp_homeAvg,first_downs_homeAvg,third_downs_opp_homeAvg,third_downs_homeAvg,fourth_downs_opp_homeAvg,fourth_downs_homeAvg,passing_yards_opp_homeAvg,passing_yards_homeAvg,rushing_yards_opp_homeAvg,rushing_yards_homeAvg,total_yards_opp_homeAvg,total_yards_homeAvg,sacks_opp_homeAvg,sacks_homeAvg,rushing_attempts_opp_homeAvg,rushing_attempts_homeAvg,fumbles_opp_homeAvg,fumbles_homeAvg,int_opp_homeAvg,int_homeAvg,turnovers_opp_homeAvg,turnovers_homeAvg,penalties_opp_homeAvg,penalties_homeAvg,drives_opp_homeAvg,drives_homeAvg,def_st_td_opp_homeAvg,def_st_td_homeAvg,possession_opp_homeAvg,possession_homeAvg,score_opp_homeAvg,score_homeAvg,score_diff_homeAvg,winner_homeAvg,third_downs_rate_opp_homeAvg,third_downs_attempts_opp_homeAvg,third_downs_rate_homeAvg,third_downs_attempts_homeAvg,fourth_downs_rate_opp_homeAvg,fourth_downs_attempts_opp_homeAvg,fourth_downs_rate_homeAvg,...,total_yards_opp_awayAvg,total_yards_awayAvg,sacks_opp_awayAvg,sacks_awayAvg,rushing_attempts_opp_awayAvg,rushing_attempts_awayAvg,fumbles_opp_awayAvg,fumbles_awayAvg,int_opp_awayAvg,int_awayAvg,turnovers_opp_awayAvg,turnovers_awayAvg,penalties_opp_awayAvg,penalties_awayAvg,drives_opp_awayAvg,drives_awayAvg,def_st_td_opp_awayAvg,def_st_td_awayAvg,possession_opp_awayAvg,possession_awayAvg,score_opp_awayAvg,score_awayAvg,score_diff_awayAvg,winner_awayAvg,third_downs_rate_opp_awayAvg,third_downs_attempts_opp_awayAvg,third_downs_rate_awayAvg,third_downs_attempts_awayAvg,fourth_downs_rate_opp_awayAvg,fourth_downs_attempts_opp_awayAvg,fourth_downs_rate_awayAvg,fourth_downs_attempts_awayAvg,complete_passes_opp_awayAvg,pass_completion_rate_opp_awayAvg,pass_attempt_opp_awayAvg,complete_passes_awayAvg,pass_completion_rate_awayAvg,pass_attempt_awayAvg,pressures_opp_awayAvg,pressures_awayAvg,penalties_yards_opp_awayAvg,penalties_yards_awayAvg,redzone_sucess_opp_awayAvg,redzone_rate_opp_awayAvg,redzone_attempts_opp_awayAvg,redzone_sucess_awayAvg,redzone_rate_awayAvg,redzone_attempts_awayAvg,season_awayAvg,day_number_awayAvg
194,2021-12-09,Vikings,Steelers,1,36,28,8,Y,7035,22.702313,22.058836,4.198073,5.114071,2.518146,3.12827,263.034405,263.160072,123.737322,145.186514,386.771726,408.346586,3.396059,1.249407,27.13578,29.195613,0.303511,0.476437,0.926825,0.836492,1.230336,1.312928,6.947119,7.967219,12.217127,11.986066,0.019509,0.195898,1826.93954,1808.236308,28.597293,30.273332,1.676039,0.536238,0.342447,12.222311,0.392093,12.804213,0.486647,6.741603,0.367932,...,399.964854,338.611456,2.7419,2.959841,31.610098,23.419552,0.109047,0.364917,1.198485,0.719559,1.307532,1.084476,9.091179,7.102215,11.579481,12.164101,0.192044,0.007707,1952.978698,1694.91644,30.110658,22.92688,-7.183778,0.335605,0.412237,13.80271,0.403833,13.42655,0.34086,8.230479,0.502061,6.116107,19.311344,0.599946,32.205274,25.310652,0.660378,38.463668,15.83874,19.827497,71.01209,64.272499,2.160161,0.545908,3.881197,2.112534,0.797064,2.998372,2021.0,7022.742105
195,2021-12-12,Browns,Ravens,1,24,22,2,Y,7038,20.856658,18.375714,3.927565,4.569309,3.771766,3.489648,210.502101,191.027824,133.102644,107.907557,343.604744,298.935381,2.604511,1.833765,28.897144,25.972375,0.74937,0.53892,1.358661,0.759465,2.108031,1.298385,7.085543,6.085398,11.826782,11.15667,0.0,0.399177,1805.69994,1794.30006,20.700221,17.909585,-2.790635,0.594894,0.320649,12.286862,0.361051,12.399218,0.533248,6.91798,0.311338,...,313.825256,353.917803,1.398669,4.479809,24.027407,29.01651,0.411922,0.760931,0.402338,1.119722,0.81426,1.880653,5.197044,7.94889,11.557392,12.083928,0.446926,0.0,1638.733477,1993.435542,20.209416,19.877142,-0.332274,0.34163,0.369068,12.545383,0.31104,14.06523,0.363191,9.622703,0.471859,5.163215,20.219283,0.610977,33.373703,25.808658,0.671356,38.296044,8.380061,22.520031,57.967368,75.008441,1.122247,0.630183,1.617068,1.791051,0.550309,3.359009,2021.0,7023.316545
196,2021-12-12,Titans,Jaguars,1,20,0,20,Y,7038,16.994076,20.678491,4.

In [35]:
x_train = get_next_week_x_train(previous_games, next_homes, next_aways)

/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:442: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:442: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:442: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_home = data_home.append(important_home[team])
/Users/evanm/Classes/Projects/nfl_prediction/nfl_data_functions_evan.py:442: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

In [39]:
x_train

,first_downs_opp_homeAvg,first_downs_homeAvg,third_downs_opp_homeAvg,third_downs_homeAvg,fourth_downs_opp_homeAvg,fourth_downs_homeAvg,passing_yards_opp_homeAvg,passing_yards_homeAvg,rushing_yards_opp_homeAvg,rushing_yards_homeAvg,total_yards_opp_homeAvg,total_yards_homeAvg,sacks_opp_homeAvg,sacks_homeAvg,rushing_attempts_opp_homeAvg,rushing_attempts_homeAvg,fumbles_opp_homeAvg,fumbles_homeAvg,int_opp_homeAvg,int_homeAvg,turnovers_opp_homeAvg,turnovers_homeAvg,penalties_opp_homeAvg,penalties_homeAvg,drives_opp_homeAvg,drives_homeAvg,def_st_td_opp_homeAvg,def_st_td_homeAvg,possession_opp_homeAvg,possession_homeAvg,score_opp_homeAvg,score_homeAvg,score_diff_homeAvg,winner_homeAvg,third_downs_rate_opp_homeAvg,third_downs_attempts_opp_homeAvg,third_downs_rate_homeAvg,third_downs_attempts_homeAvg,fourth_downs_rate_opp_homeAvg,fourth_downs_attempts_opp_homeAvg,fourth_downs_rate_homeAvg,fourth_downs_attempts_homeAvg,complete_passes_opp_homeAvg,pass_completion_rate_opp_homeAvg,pass_attempt_opp_homeAvg,complete_passes_homeAvg,pass_completion_rate_homeAvg,pass_attempt_homeAvg,pressures_opp_homeAvg,pressures_homeAvg,...,total_yards_opp_awayAvg,total_yards_awayAvg,sacks_opp_awayAvg,sacks_awayAvg,rushing_attempts_opp_awayAvg,rushing_attempts_awayAvg,fumbles_opp_awayAvg,fumbles_awayAvg,int_opp_awayAvg,int_awayAvg,turnovers_opp_awayAvg,turnovers_awayAvg,penalties_opp_awayAvg,penalties_awayAvg,drives_opp_awayAvg,drives_awayAvg,def_st_td_opp_awayAvg,def_st_td_awayAvg,possession_opp_awayAvg,possession_awayAvg,score_opp_awayAvg,score_awayAvg,score_diff_awayAvg,winner_awayAvg,third_downs_rate_opp_awayAvg,third_downs_attempts_opp_awayAvg,third_downs_rate_awayAvg,third_downs_attempts_awayAvg,fourth_downs_rate_opp_awayAvg,fourth_downs_attempts_opp_awayAvg,fourth_downs_rate_awayAvg,fourth_downs_attempts_awayAvg,complete_passes_opp_awayAvg,pass_completion_rate_opp_awayAvg,pass_attempt_opp_awayAvg,complete_passes_awayAvg,pass_completion_rate_awayAvg,pass_attempt_awayAvg,pressures_opp_awayAvg,pressures_awayAvg,penalties_yards_opp_awayAvg,penalties_yards_awayAvg,redzone_sucess_opp_awayAvg,redzone_rate_opp_awayAvg,redzone_attempts_opp_awayAvg,redzone_sucess_awayAvg,redzone_rate_awayAvg,redzone_attempts_awayAvg,season_awayAvg,day_number_awayAvg
0,23.05347,22.088254,4.297109,5.171106,2.777219,2.192405,260.051607,286.740108,132.605983,96.77977,392.657589,383.519879,2.594089,1.87411,28.70367,25.79342,0.455266,0.714655,0.890238,0.254738,1.345504,0.969393,5.420679,6.450829,11.32569,11.479099,0.029264,0.293846,1854.40931,1798.354463,28.89594,27.409998,-1.485941,0.304357,0.359824,11.833466,0.409568,12.20632,0.22997,9.112404,0.373327,4.43202,22.069552,0.629233,35.300225,25.348914,0.682342,36.918102,19.286796,12.077461,...,370.947281,320.417185,4.11285,1.939762,29.415147,23.129328,0.16357,0.547376,0.797727,0.579338,0.961298,1.126714,8.136768,5.653322,10.869222,11.246152,0.288066,0.011561,2015.468046,1656.374661,27.165986,20.39032,-6.775667,0.503407,0.439784,13.704065,0.451903,13.639825,0.332718,5.345719,0.253091,8.17416,21.967016,0.674112,32.807911,23.965977,0.640567,37.695501,23.75811,10.241246,55.518135,53.908748,2.240241,0.485529,4.321795,1.6688,0.820597,2.497559,2021.0,7016.613157
1,20.784986,17.563571,5.391347,3.853963,4.157649,2.234472,180.253151,191.541737,140.653965,111.861336,320.907117,303.403072,1.906767,2.750648,31.845716,24.458563,0.124055,0.808379,2.037991,0.639198,2.162046,1.447577,5.628314,7.128097,11.240173,10.735005,0.0,0.098765,1923.549911,1676.450089,20.050331,14.864378,-5.185953,0.392341,0.439307,12.430292,0.310807,12.098828,0.424872,8.376969,0.236237,5.741531,20.402479,0.671804,30.499156,17.515984,0.556688,31.905022,10.855425,19.863385,...,325.737884,336.376704,2.098003,4.719713,21.54111,32.024765,0.617882,0.141397,0.103507,1.679582,0.72139,1.820979,5.795565,6.923335,11.336088,11.625893,0.17039,0.0,1443.100216,2205.153313,18.314123,18.815712,0.501589,0.512444,0.322832,12.318075,0.424893,15.097845,0.314017,7.934055,0.332789,5.744822,19.328925,0

In [36]:
df_X = x_train.drop(['day_number_awayAvg','season_homeAvg','day_number_homeAvg','season_awayAvg'], axis=1)
results_dt = tune_dt_grid(X_train_full, y_train_full, X_test, y_test, criterion=['gini', 'entropy'], splitter=['best', 'random'], max_depth=[None, 1, 5, 10], min_samples_split=[2, 10], min_samples_leaf=[1, 2, 4])
from sklearn.tree import DecisionTreeClassifier
params = results_dt["best_dt_params"]
clf = DecisionTreeClassifier(criterion=params['criterion'], splitter=params['splitter'], max_depth=params['max_depth'], min_samples_split=params['min_samples_split'], min_samples_leaf=params['min_samples_leaf'])
clf.fit(X_train_full, y_train_full)
1-np.mean(abs(testy_x.winner - clf.predict(df_X)))

/var/folders/40/wpjlj6d57pz0lygq322vmsyc0000gn/T/ipykernel_13899/113238962.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_X = x_train.drop(['day_number_awayAvg','season_homeAvg','day_number_homeAvg','season_awayAvg'], axis=1)


Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.0s
[CV]

DecisionTreeClassifier(min_samples_leaf=2)

In [37]:
1-np.mean(abs(testy_x.winner - clf.predict(df_X)))

/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/evanm/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


0.7142857142857143